In [1]:
import pandas as pd

In [2]:
total = pd.read_csv('./기상자료(2013.01.01~2018.12.31)/jupyter2/2013-2018년_가스공급량과_기온.csv', encoding='utf-8')
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368088 entries, 0 to 368087
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  368088 non-null  int64  
 1   연월일         368088 non-null  object 
 2   시간          368088 non-null  int64  
 3   구분          368088 non-null  int64  
 4   공급량         368088 non-null  float64
 5   year        368088 non-null  int64  
 6   month       368088 non-null  int64  
 7   day         368088 non-null  int64  
 8   weekday     368088 non-null  int64  
 9   기온          366961 non-null  float64
dtypes: float64(2), int64(7), object(1)
memory usage: 28.1+ MB


In [3]:
total.head()

,Unnamed: 0,연월일,시간,구분,공급량,year,month,day,weekday,기온
0,0,2013-01-01,1,0,2497.129,2013,1,1,1,-6.4
1,1,2013-01-01,1,1,2169.093,2013,1,1,1,-6.4
2,2,2013-01-01,1,2,226.178,2013,1,1,1,-6.4
3,3,2013-01-01,1,3,1434.516,2013,1,1,1,-6.4
4,4,2013-01-01,1,4,3272.837,2013,1,1,1,-6.4


In [4]:
sub = pd.read_csv('./data/sample_submission.csv')
sub.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,0
1,2019-01-01 02 A,0
2,2019-01-01 03 A,0
3,2019-01-01 04 A,0
4,2019-01-01 05 A,0


In [5]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120 entries, 0 to 15119
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   일자|시간|구분  15120 non-null  object
 1   공급량       15120 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 236.4+ KB


In [15]:
sub['일자|시간|구분'].str.split(r'-| ').tolist()[:6]

[['2019', '01', '01', '01', 'A'],
 ['2019', '01', '01', '02', 'A'],
 ['2019', '01', '01', '03', 'A'],
 ['2019', '01', '01', '04', 'A'],
 ['2019', '01', '01', '05', 'A'],
 ['2019', '01', '01', '06', 'A']]

In [16]:
sub[['year', 'month', 'day', '시간', '구분']] =  sub['일자|시간|구분'].str.split(r'-| ').tolist()
sub.head()

,일자|시간|구분,공급량,year,month,day,시간,구분
0,2019-01-01 01 A,0,2019,01,01,01,A
1,2019-01-01 02 A,0,2019,01,01,02,A
2,2019-01-01 03 A,0,2019,01,01,03,A
3,2019-01-01 04 A,0,2019,01,01,04,A
4,2019-01-01 05 A,0,2019,01,01,05,A


In [17]:
sub['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [18]:
# 구분 열을 숫자로 바꾸기
d_map = {}
for i, d in enumerate(sub['구분'].unique()):
    d_map[d] = i
sub['구분'] = sub['구분'].map(d_map)

In [19]:
sub

,일자|시간|구분,공급량,year,month,day,시간,구분
0,2019-01-01 01 A,0,2019,01,01,01,0
1,2019-01-01 02 A,0,2019,01,01,02,0
2,2019-01-01 03 A,0,2019,01,01,03,0
3,2019-01-01 04 A,0,2019,01,01,04,0
4,2019-01-01 05 A,0,2019,01,01,05,0
...,...,...,...,...,...,...,...
15115,2019-03-31 20 H,0,2019,03,31,20,6
15116,2019-03-31 21 H,0,2019,03,31,21,6
15117,2019-03-31 22 H,0,2019,03,31,22,6
15118,2019-03-31 23 H,0,2019,03,31,23,6


In [ ]:
# total 기온 결측값 처리


## 2019년1-3월 기온 예측

In [20]:
# 훈련 컬럼 total : year, month, day, 시간, 구분, 기온
# 입력 컬럼 sub : year, month, day, 시간, 구분, 기온
# 결과 sub : 기온

In [23]:
# 데이터 분할, 모델 라이브러리 로드
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression

In [24]:
# 훈련, 테스트 셋 설정
sel = ['year', 'month', 'day', '시간', '구분']
X = total[sel]
y = total['기온']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)